In [1]:
import queue 
import threading
import random
import time # for sleep
import datetime # for getting current time
import urllib.request # for fetching the content by hitting specific url
from bs4 import BeautifulSoup # for parsing of the content got bby hitting a specific url
import urllib.robotparser # for parsing robot.txt file and knowing which urls are acessible
import io # for file opening


# import os 



In [2]:
# Crawler Parameters
BACKQUEUES= 3
THREADS= BACKQUEUES*3
FRONTQUEUES= 5
delay_seconds=20  # wait 20 seconds before fetching URLS from 
no_of_urls_processed=0 # to how many urls are currently being crawled
exitFlag=False # to tell when all threads will exit
maximum_urls_to_be_explored=1000 # to tell how many urls will be put in frontier before finishing
pRioRity_Range=5 # how many front queues in frontier as one priority is assigned to one front queue
queueLock = threading.Lock()

# Add any other global parameters here

In [3]:
class url_with_domain:
    def __init__(self,URL):
        self.url=URL
        self.domain=self._function_to_extract_domain()
        self.robot_txt_url = "https://"+self.domain+"/robots.txt"
        print("Main Url = ",self.url)
        print("domain = ",self.domain)
        print("domain of robot.txt = ",self.robot_txt_url)
                
    def _function_to_extract_domain(self):
        dummy_url_without_https=self.url.split("https://")
        extracted_domain=dummy_url_without_https[1].split("/")
        
        return extracted_domain[0]

# FRONTIER

Frontier should use the Mercator frontier design as discussed in lecture.

Preferably it should be a class and should have the given functions.

*prioritizer* function is a stub right now, it will return a random number  between 1 to f for given URL 

In [4]:
class frontier:
        def __init__(self,n):
            self.no_of_items_in_all_queus=n
            self.F_q1=queue.Queue(n)
            self.F_q2=queue.Queue(n)
            self.F_q3=queue.Queue(n)
            self.F_q4=queue.Queue(n)
            self.F_q5=queue.Queue(n)
            
            #   for keeping the track of which back queue can get which domain
            self.back_queue_domains=["N"]*3
            
            # timer array keeping time_stamp for every back_queues
            self.back_queue_timer_array=[None]*3
            
            # to keep the track of which urls have been explored
            global maximum_urls_to_be_explored
            self.list_of_fethched_urls=[None]* maximum_urls_to_be_explored
            
            # initially giving the time stamp(earliest time the request can be made)
            #with difference of one second to successive queues 
            abc=0
            for abc in range(3):
                self.back_queue_timer_array[abc]=datetime.datetime.now()
#                 print(back_queue_timer_array[i])
                abc=abc+1
                time.sleep(1)
            
            
            self.b_q1=queue.Queue(n)
            self.b_q2=queue.Queue(n)
            self.b_q3=queue.Queue(n)
            
            self.seed_urls=["https://docs.oracle.com/en/","https://www.oracle.com/corporate/",
                       "https://en.wikipedia.org/wiki/Machine_learning","https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html"
                      ,"https://docs.oracle.com/middleware/jet210/jet/index.html",
                      "https://en.wikipedia.org/w/api.php","https://en.wikipedia.org/api/",
                       "https://en.wikipedia.org/wiki/Weka_(machine_learning)"]
            
        def check_time_stamp(self):
            back_queue_number=0
            if(self.back_queue_timer_array[0]<self.back_queue_timer_array[1] and self.back_queue_timer_array[0]<self.back_queue_timer_array[2]):
                back_queue_number=1
            elif(self.back_queue_timer_array[1]<self.back_queue_timer_array[0] and self.back_queue_timer_array[1]<self.back_queue_timer_array[2]):
                back_queue_number=2
            elif(self.back_queue_timer_array[2]<self.back_queue_timer_array[0] and self.back_queue_timer_array[2]<self.back_queue_timer_array[1]):
                back_queue_number=3
            return back_queue_number
        def get_seed_urls_list(self):
            return self.seed_urls
        
        def prioritizer(self,URL,f):
            return random.randint(1,f)
        
        def assigning_url_to_front_queue(self,URL,f):
            if(f==1):
                if(self.F_q1.full()!=True):
                    URL_with_domain_seprated=url_with_domain(URL)
                    self.F_q1.put(URL_with_domain_seprated)
            elif(f==2):
                if(self.F_q2.full()!=True):
                    URL_with_domain_seprated=url_with_domain(URL)
                    self.F_q2.put(URL_with_domain_seprated)
            elif(f==3):
                if(self.F_q3.full()!=True):
                    URL_with_domain_seprated=url_with_domain(URL)
                    self.F_q3.put(URL_with_domain_seprated)          
            elif(f==4):
                if(self.F_q4.full()!=True):
                    URL_with_domain_seprated=url_with_domain(URL)
                    self.F_q4.put(URL_with_domain_seprated)
            elif(f==5):
                if(self.F_q5.full()!=True):
                    URL_with_domain_seprated=url_with_domain(URL)
                    self.F_q5.put(URL_with_domain_seprated)
                    
        def assigning_url_to_back_queue(self):
            poped_object=url_with_domain("https://dummy_domain/asdj/dskasjd")
            if(self.b_q1.empty()==True or self.b_q2.empty()==True or self.b_q3.empty()==True  ):
                random_no=random.randint(1,100)
                print("random_no == ",random_no)
                if(random_no>=1 and random_no<=40 and self.F_q1.empty()==False ):
                    poped_object=self.F_q1.get()
                elif (random_no>=41 and random_no<=70 and self.F_q2.empty()==False):
                    poped_object=self.F_q2.get()
                elif (random_no>=71 and random_no<=85 and self.F_q3.empty()==False):
                    poped_object=self.F_q3.get()
                elif (random_no>=86 and random_no<=95 and self.F_q4.empty()==False):
                    poped_object=self.F_q4.get()
                elif (random_no>=96 and random_no<=100 and self.F_q5.empty()==False):
                    poped_object=self.F_q5.get()
                    
                extracter_domain_of_URL_popped_from_F_Queue=poped_object._function_to_extract_domain()
                # to check whether the domain of popped url from front queue matches the 
                # domain of back queue 1
#                 print(self.back_queue_domains)
                if( self.back_queue_domains[0] == extracter_domain_of_URL_popped_from_F_Queue):
                        self.b_q1.put(poped_object)
                # to check whether the domain of popped url from front queue matches the 
                # domain of back queue 2
                elif( self.back_queue_domains[1] == extracter_domain_of_URL_popped_from_F_Queue):
                        self.b_q2.put(poped_object)
                # to check whether the domain of popped url from front queue matches the 
                # domain of back queue 3
                elif( self.back_queue_domains[2] == extracter_domain_of_URL_popped_from_F_Queue):
                        self.b_q3.put(poped_object)
                        
                #  if first back queue is empty
                elif(self.b_q1.empty()==True and (extracter_domain_of_URL_popped_from_F_Queue!="dummy_domain") ):
                    self.b_q1.put(poped_object)
                    self.back_queue_domains[0]=poped_object._function_to_extract_domain()

                
                #  if second back queue is empty
                elif(self.b_q2.empty()==True and (extracter_domain_of_URL_popped_from_F_Queue!="dummy_domain") ):
                    self.b_q2.put(poped_object)
                    self.back_queue_domains[1]=poped_object._function_to_extract_domain()

                #   if third back queue is empty
                elif(self.b_q3.empty()==True and (extracter_domain_of_URL_popped_from_F_Queue!="dummy_domain") ):                
                    self.b_q3.put(poped_object)
                    self.back_queue_domains[2]=poped_object._function_to_extract_domain()

# Fetching and Parsing the content of Url got from frontier 

In [5]:
  
#fetching module and parsing module combined 
    
def _function_to_extract_domain_same_function(url):
    dummy_url_without_https=url.split("https://")
    extracted_domain=dummy_url_without_https[1].split("/")
        
    return extracted_domain[0]

# function for saving the content recieved by hitting every url respectively
# it will be stored in txt file 
# with the first line containing the url which is hit to get the content 
# and the rest of line will contain the content
# Moreover the name of the file would be the serial number of the url that is being crawled
def creation_of_database(UrL,n_urls_processed,soup):
    filename = str(n_urls_processed) + ".txt"
    s=soup.get_text()
    with io.open(filename, "w", encoding="utf-8") as f:
        f.write(UrL)
        f.write(s)  
        f.close() #closing the file
    
def fetching_content_by_hitting_url(UrL,n_urls_processed):

    link_list=[None]*5000
    n_links_processed=0
    fhand= urllib.request.urlopen(UrL)
    domain = _function_to_extract_domain_same_function(UrL)
    # print("domain == ",domain)
    # for line in fhand:
    #     print(line.decode().strip())

    # print()
    # print("____________________________________________________________________________________")
    # print()
    data = fhand.read()

    # print(data.title)
    # print(data)

    soup = BeautifulSoup(data, 'html.parser')
    # print(soup.prettify())
    # if the file is of not the type xml
    if (soup.find("xml")==None):
        
        creation_of_database(UrL,n_urls_processed,soup)
#         i=0
        for link in soup.find_all( 'link' ):
            actual_link=link.get('href')
            if(actual_link.find(".css")==-1 and actual_link.find(".js")==-1 and actual_link.find(".png")==-1 ):
                # for checking if the link is relative
                if (actual_link.find("https://")==-1):

                    actual_link="https://"+ domain + actual_link # making the link complete
                    link_list[n_links_processed]=actual_link
                    n_links_processed = n_links_processed + 1
#                     print(i,"    ",actual_link)
                else:
                    link_list[n_links_processed]=actual_link
                    n_links_processed = n_links_processed + 1
#                     print(i,"    without appending",actual_link)
#             i=i+1
        # for keeping a check for the end of elements in the below list of urls
        link_list[n_links_processed]="-1"

    return link_list

# FILTER URLS
Filter the URLS that are in robots.txt files of server and the have been already processed.

In [6]:

# url filtering module on the basis of robot.txt file


def to_check_which_urls_are_already_crawled(list_Of_Filtered_urls, no_of_urls_crawled,list_of_crawled_urls):
    ultimate_filtered_list = [None]*(len(list_Of_Filtered_urls))
    
    outer_iter=0
    inner_iter=0
    ultimate_Filter_iter=0
    check=False
    while (outer_iter < (len(list_Of_Filtered_urls))):
        inner_iter = 0
        while (inner_iter <= no_of_urls_crawled):
            if(list_Of_Filtered_urls[outer_iter]==list_of_crawled_urls[inner_iter]):
                check=True
            inner_iter = inner_iter + 1
            
        if(check==False):
            ultimate_filtered_list[ultimate_Filter_iter] = list_Of_Filtered_urls[outer_iter]
            ultimate_Filter_iter = ultimate_Filter_iter + 1
        else:
            check=False
        outer_iter = outer_iter + 1
    ultimate_filtered_list[ultimate_Filter_iter] = "-1"
    return ultimate_filtered_list
    

def function_for_url_filtering_on_the_basis_of_robot_file(list_of_fetched_urls,Robot_txt_URL):
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(Robot_txt_URL)
    rp.read()

    # rrate = rp.request_rate("*")
    # print(" rrate.requests == ",rrate.requests )
    # print(" rrate.seconds == ",rrate.seconds ) 
    # print (" rp.crawl_delay == ",rp.crawl_delay("*") )
    
    # loop to know how many urls in the fetched list
    no_of_items_in_list_fetched=0
    while (list_of_fetched_urls[no_of_items_in_list_fetched] != "-1"):
        no_of_items_in_list_fetched = no_of_items_in_list_fetched + 1
    
    filtered_url_list = [None] * no_of_items_in_list_fetched # list which will contain the filtered urls
    iter_filter_list= 0 # iterator pf filtered list
    ghj=0
    while (list_of_fetched_urls[ghj] != "-1" ):
        url_check = str(list_of_fetched_urls[ghj])
        if(rp.can_fetch("*", url_check )==True):
            filtered_url_list[iter_filter_list] = list_of_fetched_urls[ghj]
            iter_filter_list = iter_filter_list + 1
        ghj = ghj +1
    
    return filtered_url_list



# --Write any other codes/data require to run the crawler above this cell------

# Run Crawler

In [7]:
import threading

list_of_URLS_returned_by_fetch_and_parse_module=[None]*5000

class myThread (threading.Thread):
    def __init__(self, threadID, name, f):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.Frontier = f
        self.URL=[None]*1
    def run(self):
        global no_of_urls_processed
        print ("Starting " , self.name)
        request_url(self.name, self.Frontier,self.URL[0],no_of_urls_processed)
        print ("Exiting " , self.name)
        





def request_url(threadName, Frontier,URL,no_of_urls_processed):
    Back_queue_no=0
    global exitFlag,delay_seconds,pRioRity_Range,list_of_URLS_returned_by_fetch_and_parse_module,queueLock
    while (exitFlag==False):
        queueLock.acquire()
        if ( Frontier.b_q1.empty()==False or Frontier.b_q2.empty()==False or Frontier.b_q3.empty()==False ):
            Back_queue_no=Frontier.check_time_stamp()
            if(Back_queue_no == 1): # means first back queue time stamp is least
                current_time=datetime.datetime.now()
                if(Frontier.back_queue_timer_array[0]<=current_time):#means the time stamp of first back queue is less or
                                                                            # equal to current time, hence thread can get url
                                                                            # from it.
                    URL=Frontier.b_q1.get()
                                                            
                    # __________________for keeping the track of how many urls have been processed______________ #
                    Frontier.list_of_fethched_urls[no_of_urls_processed]= URL.url                    
                    no_of_urls_processed = no_of_urls_processed + 1
                    
                    #____hitting the url and getting more url by parsing the content got by hitting this urls___#
                    list_of_URLS_returned_by_fetch_and_parse_module= fetching_content_by_hitting_url(URL.url, ( no_of_urls_processed-1) )
                    
                    #___________Url filtering on the basis of robot_txt_file___________________________________#
                    list_of_filtered_urls=function_for_url_filtering_on_the_basis_of_robot_file(list_of_URLS_returned_by_fetch_and_parse_module,URL.robot_txt_url)
                    
                    #__________for checking of duplicate url comes that is already being crawled______________#
                    Ultimate_Filtered_list_of_urls  = to_check_which_urls_are_already_crawled(list_of_filtered_urls, (no_of_urls_processed - 1) ,Frontier.list_of_fethched_urls)
                    
                    
                    #__________putting_filterd_urls_in_front_queue_________________________________________#
                    you=0
                    while(Ultimate_Filtered_list_of_urls[you]!="-1"):
                        Prioority=Frontier.prioritizer(Ultimate_Filtered_list_of_urls[you],pRioRity_Range)
                        Frontier.assigning_url_to_front_queue(Ultimate_Filtered_list_of_urls[you],Prioority)
                                                            
                    
                    # ________________ Module to check wether after the thread have taken Url from back queue 1 
                    # whether the back queue 1 gets empty if so then keep on trying to take some url to the 
                    # back queue 1 until it no Longer remains empty _____________________________________________ #
                    while(Frontier.b_q1.empty()==True):
                        Frontier.assigning_url_to_back_queue()
                    #________________________________________________________________________________________#
                    
                    print("URl is popped from Back Queue 1.")
                    print("Url == ",URL.url)
                    print("Url domain == ",URL.domain)
                    Frontier.back_queue_timer_array[0]= datetime.datetime.now() + datetime.timedelta(0,delay_seconds) # adding certain seconds
                    
                    queueLock.release()
                else:
                    queueLock.release()
            elif (Back_queue_no == 2): # means second back queue time stamp is least
                current_time=datetime.datetime.now()
                if(Frontier.back_queue_timer_array[1]<=current_time):#means the time stamp of second back queue is less or
                                                                            # equal to current time, hence thread can get url
                                                                            # from it.
                    URL=Frontier.b_q2.get()
                    
                    # __________________for keeping the track of how many urls have been processed______________ #
                    Frontier.list_of_fethched_urls[no_of_urls_processed]= URL.url                    
                    no_of_urls_processed = no_of_urls_processed + 1
                    
                    #____hitting the url and getting more url by parsing the content got by hitting this urls___#
                    list_of_URLS_returned_by_fetch_and_parse_module= fetching_content_by_hitting_url(URL.url,  ( no_of_urls_processed-1) )
                    
                    #___________Url filtering on the basis of robot_txt_file___________________________________#
                    list_of_filtered_urls=function_for_url_filtering_on_the_basis_of_robot_file(list_of_URLS_returned_by_fetch_and_parse_module,URL.robot_txt_url)
                    
                    #__________for checking of duplicate url comes that is already being crawled______________#
                    Ultimate_Filtered_list_of_urls  = to_check_which_urls_are_already_crawled(list_of_filtered_urls, (no_of_urls_processed - 1) ,Frontier.list_of_fethched_urls)
                    
                    
                    #__________putting_filterd_urls_in_front_queue_________________________________________#
                    you=0
                    while(Ultimate_Filtered_list_of_urls[you]!="-1"):
                        Prioority=Frontier.prioritizer(Ultimate_Filtered_list_of_urls[you],pRioRity_Range)
                        Frontier.assigning_url_to_front_queue(Ultimate_Filtered_list_of_urls[you],Prioority)
                    
                    # ________________ Module to check wether after the thread have taken Url from back queue 2 
                    # whether the back queue 2 gets empty if so then keep on trying to take some url to the 
                    # back queue 2 until it no Longer remains empty _____________________________________________ #
                    while(Frontier.b_q2.empty()==True):
                        Frontier.assigning_url_to_back_queue()
                    #________________________________________________________________________________________#
                    
                    print("URl is popped from Back Queue 2.")
                    print("Url == ",URL.url)
                    print("Url domain == ",URL.domain)
                    Frontier.back_queue_timer_array[1]= datetime.datetime.now() + datetime.timedelta(0,delay_seconds) # adding certain seconds
                    
                    queueLock.release()
                else:
                    queueLock.release()
            elif (Back_queue_no == 3): # means third back queue time stamp is least
                current_time=datetime.datetime.now()
                if(Frontier.back_queue_timer_array[2]<=current_time):#means the time stamp of first back queue is less or
                                                                            # equal to current time, hence thread can get url
                                                                            # from it.
                    URL=Frontier.b_q3.get()
                                                            
                    # __________________for keeping the track of how many urls have been processed______________ #
                    Frontier.list_of_fethched_urls[no_of_urls_processed]= URL.url                    
                    no_of_urls_processed = no_of_urls_processed + 1
                    
                    #____hitting the url and getting more url by parsing the content got by hitting this urls___#
                    list_of_URLS_returned_by_fetch_and_parse_module= fetching_content_by_hitting_url(URL.url ,  ( no_of_urls_processed-1) )
                    
                    #___________Url filtering on the basis of robot_txt_file___________________________________#
                    list_of_filtered_urls=function_for_url_filtering_on_the_basis_of_robot_file(list_of_URLS_returned_by_fetch_and_parse_module,URL.robot_txt_url)
                    
                    #__________for checking of duplicate url comes that is already being crawled______________#
                    Ultimate_Filtered_list_of_urls  = to_check_which_urls_are_already_crawled(list_of_filtered_urls, (no_of_urls_processed - 1) ,Frontier.list_of_fethched_urls)
                    
                    
                    #__________putting_filterd_urls_in_front_queue_________________________________________#
                    you=0
                    while(Ultimate_Filtered_list_of_urls[you]!="-1"):
                        Prioority=Frontier.prioritizer(Ultimate_Filtered_list_of_urls[you],pRioRity_Range)
                        Frontier.assigning_url_to_front_queue(Ultimate_Filtered_list_of_urls[you],Prioority)
                                        
                                        
                    # ________________ Module to check wether after the thread have taken Url from back queue 3 
                    # whether the back queue 3 gets empty if so then keep on trying to take some url to the 
                    # back queue 3 until it no Longer remains empty _____________________________________________ #
                    while(Frontier.b_q3.empty()==True):
                        Frontier.assigning_url_to_back_queue()
                    #________________________________________________________________________________________#
                    
                    print("URl is popped from Back Queue 3.")
                    print("Url == ",URL.url)
                    print("Url domain == ",URL.domain)
                    Frontier.back_queue_timer_array[2]= datetime.datetime.now() + datetime.timedelta(0,delay_seconds) # adding certain seconds
                    
                    queueLock.release()
                else:
                    queueLock.release()
            else:
                queueLock.release()
        else:
            queueLock.release()
        time.sleep(1)

# Main from where object where thread will be created and started and then populate frontier by crawling over the fetched Urls

In [ ]:
global no_of_urls_processed,exitFlag,queueLock

threadList = ["Thread-1", "Thread-2", "Thread-3", "Thread-4", "Thread-5", "Thread-6","Thread-7", "Thread-8", "Thread-9"]
nameList = ["One", "Two", "Three", "Four", "Five"]


# workQueue = Queue.Queue(10)
threads = []
threadID = 1
Frontier_1=frontier(100)

# Create new threads
for tName in threadList:
    thread = myThread(threadID, tName, Frontier_1)
    thread.start()
    threads.append(thread)
    threadID += 1
        
# Fill the queue
queueLock.acquire()

seed_urls=Frontier_1.get_seed_urls_list()
ii=1
for seed_url in seed_urls:
    priority=Frontier_1.prioritizer(seed_url,pRioRity_Range)
    print(ii,"    Priority of seed url ",seed_url,"  == ",priority)
    Frontier_1.assigning_url_to_front_queue(seed_url,priority)
#     Frontier_1.assigning_url_to_back_queue()
    ii=ii+1
            
kk=0
for kk in range(10):
    Frontier_1.assigning_url_to_back_queue()
    kk=kk+1

    
queueLock.release()

# Wait for queue to empty
while (no_of_urls_processed<=10):
    pass

# Notify threads it's time to exit
exitFlag = True

# Wait for all threads to complete
for t in threads:
    t.join()
print ("Exiting Main Thread")

Starting Starting   Thread-1
Starting  Thread-3
Thread-2
Starting  Thread-4Starting  Thread-5
Starting 
 Thread-6
Starting  Thread-7
Starting  Thread-8
Starting  Thread-9


# ------------------------------------------------------End of Notebook-----------------------------